In [ ]:

#por mientras porque no puedo hacer fetch de binance en el servidor de usa 


In [72]:

#get the now time in this format "2025-09-14T03:15:00Z"

from datetime import datetime, timezone

now_iso = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
print(now_iso)


2025-09-16T02:02:11Z


In [16]:

#minimun working code
import ccxt

exchange = ccxt.coinbase()  # create client
symbol = "BTC/USD"
start_time = exchange.parse8601("2025-09-15T20:54:06Z")  # ISO → ms
# fetch one-minute candles
ohlcv = exchange.fetch_ohlcv(symbol, '1m', start_time, limit=300)

# print 
print(ohlcv)


[[1757969700000, 115312.04, 115338.98, 115307.11, 115338.98, 2.01295954], [1757969760000, 115338.97, 115339.99, 115324.03, 115328.03, 3.12677329], [1757969820000, 115328.02, 115328.03, 115328.02, 115328.03, 0.01906512]]


In [ ]:
import ccxt
import time
#
# Initialize the exchange client
exchange = ccxt.coinbase()

# Parameters
symbol = "BTC/USD"
start_time = exchange.parse8601("2025-09-15T17:06:00Z")  # Start time in ISO format (converted to ms)

# Get the current time
current_time = exchange.milliseconds()  # Current time in milliseconds

In [ ]:
# Function to calculate the difference between start_time and current_time in minutes
def get_time_diff_in_minutes(start_time, current_time):
    return (current_time - start_time) // 60000  # Difference in minutes

# Function to fetch data for a given time window
def fetch_data(symbol, start_time, limit=300):
    ohlcv_data = exchange.fetch_ohlcv(symbol, '1m', start_time, limit=limit)
    return ohlcv_data


def fetch_main(symbol, start_time, current_time, time_diff):
    # If the time difference exceeds 300 minutes, fetch in multiple calls
    if time_diff > 300:
        ohlcv_data = []
        while start_time < current_time:
            # Fetch data for a chunk (300 candles per request)
            ohlcv_chunk = fetch_data(symbol, start_time, 300)
            ohlcv_data.extend(ohlcv_chunk)
    
            # Update the start_time for the next request (next minute after the last fetched data)
            if ohlcv_chunk:
                last_timestamp = ohlcv_chunk[-1][0]
                start_time = last_timestamp + 1  # Set the next start time to one minute after the last candle
    
            # Sleep for a second to avoid hitting the rate limit
            time.sleep(1)
    else:
        # If the time span is less than or equal to 300 minutes, fetch data once
        ohlcv_data = fetch_data(symbol, start_time, 300)
    
    # Print the fetched data
    return ohlcv_data

# Fetch the data based on time difference
time_diff = get_time_diff_in_minutes(start_time, current_time)
ohlcv_data = fetch_main(symbol, start_time, current_time, )

In [ ]:
def verify_data_complete(ohlcv_data, start_time, current_time, tolerance_ms=50000):
    # Check if ohlcv_data is not empty
    if not ohlcv_data:
        print("No data fetched!")
        return False
    
    # Check that data has the expected structure (i.e., timestamp, open, high, low, close, volume)
    for i, candle in enumerate(ohlcv_data):
        if len(candle) != 6:
            print(f"Invalid data at index {i}: {candle}")
            return False

    # Verify the first and last timestamps are within the expected time range
    first_timestamp = ohlcv_data[0][0]
    last_timestamp = ohlcv_data[-1][0]
    
    if first_timestamp < start_time:
        print(f"First timestamp is too early: {first_timestamp}. Expected >= {start_time}")
        return False
    if last_timestamp > current_time + tolerance_ms:
        print(f"Last timestamp is too late: {last_timestamp}. Expected <= {current_time + tolerance_ms}")
        return False
    
    # Verify that each chunk connects seamlessly to the next (no gaps between timestamps)
    for i in range(1, len(ohlcv_data)):
        prev_candle = ohlcv_data[i - 1]
        current_candle = ohlcv_data[i]
        
        # Check if the last timestamp of the previous chunk + 1 is the first timestamp of the current chunk
        if current_candle[0] != prev_candle[0] + 60000:  # 60000 ms = 1 minute
            print(f"Gap detected between timestamps: {prev_candle[0]} and {current_candle[0]}")
            return False
        
    # Verify that there are no duplicates (last timestamp of previous chunk should not be duplicated)
    seen_timestamps = set()
    for i, candle in enumerate(ohlcv_data):
        timestamp = candle[0]
        if timestamp in seen_timestamps:
            print(f"Duplicate timestamp found at index {i}: {timestamp}")
            return False
        seen_timestamps.add(timestamp)
    
    print("Data is complete and correctly ordered.")
    return True


# After fetching all the data, verify that the data is complete
data_complete = verify_data_complete(ohlcv_data, start_time, current_time)
print(data_complete)

In [95]:
import ccxt
import time
import pandas as pd
from datetime import datetime

class DataFetcher:
    def __init__(self, symbol, start_time, current_time, exchange_name, start_time_count):
        # Initialize the exchange client (e.g., Coinbase)
        self.exchange = getattr(ccxt, exchange_name)()  # Dynamically initialize the exchange
        self.symbol = symbol
        self.start_time = self.exchange.parse8601(start_time)  # Convert ISO format to ms
        self.current_time = current_time
        self.start_time_count = start_time_count

    # Function to calculate the difference between start_time and current_time in minutes
    def get_time_diff_in_minutes(self):
        return (self.current_time - self.start_time) // 60000  # Difference in minutes

    # Function to fetch data for a given time window
    def fetch_data(self, limit=300):
        ohlcv_data = self.exchange.fetch_ohlcv(self.symbol, '1m', self.start_time, limit=limit)
        return ohlcv_data

    def process_data(self, ohlcv_data):
        # Convert the OHLCV data into a DataFrame
        df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        
        # Add 'time_count' column starting from start_time_count and incrementing by 1 for each row
        df['time_count'] = range(self.start_time_count, self.start_time_count + len(df))
        
        # Add 'date_time_iso' column for human-readable date-time
        df['date_time_iso'] = df['timestamp'].apply(lambda x: datetime.utcfromtimestamp(x / 1000).strftime('%Y-%m-%d %H:%M:%S'))
        
        # Add 'date' column (just the date part of the timestamp)
        df['date'] = df['timestamp']
        
        # Reorder columns to match the desired structure
        df = df[['time_count', 'date', 'date_time_iso', 'open', 'high', 'low', 'close', 'volume']]

        return df

    # Function to fetch data in chunks or once depending on the time difference
    def fetch_main(self):
        time_diff = self.get_time_diff_in_minutes()
        ohlcv_data = []

        if time_diff > 300:
            # If the time difference exceeds 300 minutes, fetch in multiple calls
            while self.start_time < self.current_time:
                # Fetch data for a chunk (300 candles per request)
                ohlcv_chunk = self.fetch_data(300)
                ohlcv_data.extend(ohlcv_chunk)
    
                # Update the start_time for the next request (next minute after the last fetched data)
                if ohlcv_chunk:
                    last_timestamp = ohlcv_chunk[-1][0]
                    self.start_time = last_timestamp + 60000  # Set the next start time to one minute after the last candle
    
                # Sleep for a second to avoid hitting the rate limit
                time.sleep(1)
        else:
            # If the time span is less than or equal to 300 minutes, fetch data once
            ohlcv_data = self.fetch_data(300)

        df = self.process_data(ohlcv_data)

        return df


# Example Usage:
symbol = "BTC/USD"
start_time = "2025-09-16T02:01:00Z"  # ISO format for start time
current_time = int(time.time() * 1000)  # Current time in milliseconds
exchange_name = 'coinbase'
# Create an instance of DataFetcher
fetcher = DataFetcher(symbol, start_time, current_time, exchange_name, start_time_count=0)

# Fetch the data
ohlcv_data_df = fetcher.fetch_main()

# Print the fetched data
#print(ohlcv_data)


In [97]:
ohlcv_data_df

,time_count,date,date_time_iso,open,high,low,close,volume
0,0,1757988060000,2025-09-16 02:01:00,115091.43,115091.44,115073.60,115073.61,0.707373
1,1,1757988120000,2025-09-16 02:02:00,115073.62,115097.17,115057.50,115076.70,0.948681
2,2,1757988180000,2025-09-16 02:03:00,115076.70,115076.70,115050.80,115050.81,0.711833
3,3,1757988240000,2025-09-16 02:04:00,115050.81,115090.04,115050.81,115086.90,1.853703
4,4,1757988300000,2025-09-16 02:05:00,115086.90,115136.96,115073.60,115123.82,1.056595
5,5,1757988360000,2025-09-16 02:06:00,115123.82,115169.45,115123.82,115162.32,2.074357
6,6,1757988420000,2025-09-16 02:07:00,115162.32,115162.32,115135.86,115142.46,0.924116
7,7,1757988480000,2025-09-16 02:08:00,115142.46,115169.60,115122.87,115144.95,1.069064
8,8,1757988540000,2025-09-16 02:09:00,115145.07,115145.07,115124.79,115124.81,0.371751
9,9,1757988600000,2025-09-16 02:10:00,115124.81,115167.84,115115.00,115167.80,1.101591
